In [ ]:
### SVM

In [3]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from matplotlib import pyplot as plt
import seaborn as sn
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.model_selection import train_test_split

In [4]:
TRAIN_PATH = 'C:\\Users\\mychu\\Workspace\\ML_Paper\\data\\train.csv'
TEST_PATH = 'C:\\Users\\mychu\\Workspace\\ML_Paper\\data\\test.csv'
TEST_TARGET_PATH = 'C:\\Users\\mychu\\Workspace\\ML_Paper\\data\\testtarget.csv'

df = pd.read_csv(TRAIN_PATH)
df_test = pd.read_csv(TEST_PATH) # hedef sütunu içermiyor
df_correct = pd.read_csv(TEST_TARGET_PATH)

In [5]:
le = preprocessing.LabelEncoder()

In [6]:
# TRAIN

In [7]:
df_categorical = df.select_dtypes(include=['object'])
df_categorical = df_categorical.fillna("none")
for column in df_categorical:
    df_categorical[column] = le.fit_transform(df_categorical[column])

In [8]:
df_numerical = df.select_dtypes(include=['int64','float64'])

for column in df_numerical:
    df_numerical[column] = df_numerical[column].fillna(df_numerical[column].mean())

df_new = df_categorical.join(df_numerical)

c:\users\mychu\workspace\ml_paper\ml_env\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [9]:
# TEST

In [10]:
df_test_categorical = df_test.select_dtypes(include=['object'])

df_test_categorical = df_test_categorical.fillna("none")

for column in df_test_categorical:
    df_test_categorical[column] = le.fit_transform(df_test_categorical[column])

In [11]:
df_test_numerical = df_test.select_dtypes(include=['int64','float64'])

for column in df_test_numerical:
    df_test_numerical[column] = df_test_numerical[column].fillna(df_test_numerical[column].mean())

df_test_new = df_test_categorical.join(df_test_numerical)

c:\users\mychu\workspace\ml_paper\ml_env\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [12]:
corr = df.corr()
high_correlated_columns = corr[abs(corr["SalePrice"])>0.2].index
high_correlated_columns = high_correlated_columns.drop("SalePrice")

In [29]:
x_train = df_new[high_correlated_columns].values
x_test = df_test_new[high_correlated_columns].values

In [30]:
y_test = df_correct["SalePrice"]
y_train = df["SalePrice"]

In [31]:
y_train_max = max(y_train.values)
y_train_min = min(y_train.values)
y_train = (y_train-y_train_min)/(y_train_max-y_train_min)

In [32]:
y_test_max = max(y_test.values)
y_test_min = min(y_test.values)
y_test = (y_test-y_test_min)/(y_test_max-y_test_min)

In [33]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()

x_train = min_max_scaler.fit_transform(x_train)
x_test = min_max_scaler.fit_transform(x_test)

In [34]:
full = np.concatenate((y_train,y_test))
std_dev = np.std(full)

In [35]:
y_train = np.array([int((x/(std_dev*2))) for x in y_train])
y_test = np.array([int((x/(std_dev*2))) for x in y_test])

In [36]:
np.unique(y_train)

array([0, 1, 2, 3])

In [37]:
clf = svm.SVC(kernel="poly",degree=7,gamma="scale")
clf.fit(x_train, y_train)

SVC(degree=7, kernel='poly')

In [38]:
y_pred = clf.predict(x_train)

In [39]:
conf_matrix = confusion_matrix(y_train,y_pred)
print(conf_matrix)
print("Total Accuracy---------------------------------")



conf_vals = []
for i in range(conf_matrix.shape[0]):
    TP = conf_matrix[i,i]
    FP = conf_matrix[i,:].sum() - TP
    FN = conf_matrix[:,i].sum() - TP
    TN = conf_matrix.sum() - (TP + FP + FN)
    conf_vals.append([TP,FP,FN,TN])
conf_vals = np.array(conf_vals)

# Total Accuracy = (TP + TN)/(TP + TN + FP + FN)
acc = (y_train == y_pred).sum()/y_train.shape[0]
print(f"Accuracy:{acc}")
print("Accuracy---------------------------------")





# Accuracy = (TP + TN)/(TP + TN + FP + FN)
for i in range(conf_matrix.shape[0]):
    s = (conf_vals[i,0] + conf_vals[i,3]) / conf_vals[i,:].sum()
    print(f"Sınıf{i} = {s}")
    
    
    
    


print("Sensitivity------------------------------------")
# Sensitivity = TP / TP + FN
sm = 0
for i in range(conf_matrix.shape[0]):
    s = conf_vals[i,0] / (conf_vals[i,0] + conf_vals[i,2])
    if not (np.isnan(s)):
        sm += s
        
    print(f"Sınıf{i} = TP({conf_vals[i,0]}) / (TP({conf_vals[i,0]}) + FN({conf_vals[i,2]})) = {s}")
print(f"avg = {sm/conf_matrix.shape[0]}")   






print("Specificity-----------------------------------")
# Specificity = TN / TN + FP
sm = 0
for i in range(conf_matrix.shape[0]):
    s = conf_vals[i,3] / (conf_vals[i,3] + conf_vals[i,1])
    if not (np.isnan(s)):
        sm += s
    print(f"Sınıf{i} = TN({conf_vals[i,3]}) / (TN({conf_vals[i,3]})+FP({conf_vals[i,1]})) = {s}")
print(f"avg = {sm/conf_matrix.shape[0]}") 




print("Precision-----------------------------------")
##### Precision = TP / TP + FP
sm = 0
for i in range(conf_matrix.shape[0]):
    s = conf_vals[i,0] / (conf_vals[i,0] + conf_vals[i,1])
    if not (np.isnan(s)):
        sm += s
    print(f"Sınıf{i} = TP({conf_vals[i,0]}) / (TP({conf_vals[i,0]}) + FP({conf_vals[i,1]})) = {s}")
print(f"avg = {sm/conf_matrix.shape[0]}") 

[[1119    1    0    0]
 [   5  310    0    0]
 [   0    0   21    0]
 [   0    0    0    4]]
Total Accuracy---------------------------------
Accuracy:0.9958904109589041
Accuracy---------------------------------
Sınıf0 = 0.9958904109589041
Sınıf1 = 0.9958904109589041
Sınıf2 = 1.0
Sınıf3 = 1.0
Sensitivity------------------------------------
Sınıf0 = TP(1119) / (TP(1119) + FN(5)) = 0.9955516014234875
Sınıf1 = TP(310) / (TP(310) + FN(1)) = 0.9967845659163987
Sınıf2 = TP(21) / (TP(21) + FN(0)) = 1.0
Sınıf3 = TP(4) / (TP(4) + FN(0)) = 1.0
avg = 0.9980840418349716
Specificity-----------------------------------
Sınıf0 = TN(335) / (TN(335)+FP(1)) = 0.9970238095238095
Sınıf1 = TN(1144) / (TN(1144)+FP(5)) = 0.9956483899042646
Sınıf2 = TN(1439) / (TN(1439)+FP(0)) = 1.0
Sınıf3 = TN(1456) / (TN(1456)+FP(0)) = 1.0
avg = 0.9981680498570186
Precision-----------------------------------
Sınıf0 = TP(1119) / (TP(1119) + FP(1)) = 0.9991071428571429
Sınıf1 = TP(310) / (TP(310) + FP(5)) = 0.9841269841269841
S

In [40]:
y_pred = clf.predict(x_test)

In [42]:
conf_matrix = confusion_matrix(y_test,y_pred)
print(conf_matrix)
print("Total Accuracy---------------------------------")



conf_vals = []
for i in range(conf_matrix.shape[0]):
    TP = conf_matrix[i,i]
    FP = conf_matrix[i,:].sum() - TP
    FN = conf_matrix[:,i].sum() - TP
    TN = conf_matrix.sum() - (TP + FP + FN)
    conf_vals.append([TP,FP,FN,TN])
conf_vals = np.array(conf_vals)

# Total Accuracy = (TP + TN)/(TP + TN + FP + FN)
acc = (y_test == y_pred).sum()/y_train.shape[0]
print(f"Accuracy:{acc}")
print("Accuracy---------------------------------")





# Accuracy = (TP + TN)/(TP + TN + FP + FN)
for i in range(conf_matrix.shape[0]):
    s = (conf_vals[i,0] + conf_vals[i,3]) / conf_vals[i,:].sum()
    print(f"Sınıf{i} = {s}")
    
    
    
    


print("Sensitivity------------------------------------")
# Sensitivity = TP / TP + FN
sm = 0
for i in range(conf_matrix.shape[0]):
    s = conf_vals[i,0] / (conf_vals[i,0] + conf_vals[i,2])
    if not (np.isnan(s)):
        sm += s
        
    print(f"Sınıf{i} = TP({conf_vals[i,0]}) / (TP({conf_vals[i,0]}) + FN({conf_vals[i,2]})) = {s}")
print(f"avg = {sm/conf_matrix.shape[0]}")   






print("Specificity-----------------------------------")
# Specificity = TN / TN + FP
sm = 0
for i in range(conf_matrix.shape[0]):
    s = conf_vals[i,3] / (conf_vals[i,3] + conf_vals[i,1])
    if not (np.isnan(s)):
        sm += s
    print(f"Sınıf{i} = TN({conf_vals[i,3]}) / (TN({conf_vals[i,3]})+FP({conf_vals[i,1]})) = {s}")
print(f"avg = {sm/conf_matrix.shape[0]}") 




print("Precision-----------------------------------")
##### Precision = TP / TP + FP
sm = 0
for i in range(conf_matrix.shape[0]):
    s = conf_vals[i,0] / (conf_vals[i,0] + conf_vals[i,1])
    if not (np.isnan(s)):
        sm += s
    print(f"Sınıf{i} = TP({conf_vals[i,0]}) / (TP({conf_vals[i,0]}) + FP({conf_vals[i,1]})) = {s}")
print(f"avg = {sm/conf_matrix.shape[0]}") 

[[796   4   0   0]
 [272 298   0   0]
 [  8  70   0   0]
 [  1  10   0   0]]
Total Accuracy---------------------------------
Accuracy:0.7493150684931507
Accuracy---------------------------------
Sınıf0 = 0.8046607265250172
Sınıf1 = 0.7559972583961617
Sınıf2 = 0.9465387251542152
Sınıf3 = 0.9924605894448252
Sensitivity------------------------------------
Sınıf0 = TP(796) / (TP(796) + FN(281)) = 0.7390900649953575
Sınıf1 = TP(298) / (TP(298) + FN(84)) = 0.7801047120418848
Sınıf2 = TP(0) / (TP(0) + FN(0)) = nan
Sınıf3 = TP(0) / (TP(0) + FN(0)) = nan
avg = 0.37979869425931057
Specificity-----------------------------------
Sınıf0 = TN(378) / (TN(378)+FP(4)) = 0.9895287958115183
Sınıf1 = TN(805) / (TN(805)+FP(272)) = 0.7474466109563602
Sınıf2 = TN(1381) / (TN(1381)+FP(78)) = 0.9465387251542152
Sınıf3 = TN(1448) / (TN(1448)+FP(11)) = 0.9924605894448252
avg = 0.9189936803417298
Precision-----------------------------------
Sınıf0 = TP(796) / (TP(796) + FP(4)) = 0.995
Sınıf1 = TP(298) / (TP(298) 

c:\users\mychu\workspace\ml_paper\ml_env\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in longlong_scalars
